<a href="https://colab.research.google.com/github/vasukondreddy/PyTorch/blob/main/PyTorch_RNN_Question_Answer_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simple Question and Answer System Using Pytorch

In [149]:
import pandas as pd
df=pd.read_csv('/content/Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [150]:
# tokenize

def tokenize(text):
  text=text.lower()
  text=text.replace('?','')
  text=text.replace("'","")
  return text.split()

In [151]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [152]:
# vocab (how many unique words present)

vocab={'<UNK>':0} # UNK - unknown

In [173]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [174]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [175]:
# len(vocab)
vocab


{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [176]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [177]:
text_to_indices("what is RNN",vocab) # RNN not in vocab so replace with 0(UNK)

[1, 2, 0]

In [158]:
import torch
from torch.utils.data import Dataset,DataLoader

In [178]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [179]:
dataset=QADataset(df,vocab)

In [180]:
dataloader = DataLoader(dataset,batch_size=1, shuffle=True)

In [181]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([28])
tensor([[ 42, 250, 251, 118, 252, 253]]) tensor([254])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([260])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[ 10,  11, 157, 158, 159]]) tensor([160])
tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([307])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([68])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([36])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([52])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[ 42,   2,   3, 274, 211, 275]]) tensor([276])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([54])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([321])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([179])
tensor([[10, 96,  3, 97]]) tensor([98])


In [163]:
import torch.nn as nn


In [182]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output


In [184]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [186]:
learning_rate=0.001
epochs=20

In [187]:
model=SimpleRNN(len(vocab))

In [188]:
criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [189]:
# training loop
for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)
    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()
    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 524.215769
Epoch: 2, Loss: 455.578146
Epoch: 3, Loss: 379.433719
Epoch: 4, Loss: 315.714605
Epoch: 5, Loss: 263.602610
Epoch: 6, Loss: 215.133728
Epoch: 7, Loss: 170.334043
Epoch: 8, Loss: 132.305160
Epoch: 9, Loss: 101.133684
Epoch: 10, Loss: 77.901946
Epoch: 11, Loss: 60.044478
Epoch: 12, Loss: 47.217107
Epoch: 13, Loss: 37.718290
Epoch: 14, Loss: 31.510187
Epoch: 15, Loss: 26.063461
Epoch: 16, Loss: 22.004982
Epoch: 17, Loss: 18.857293
Epoch: 18, Loss: 16.378168
Epoch: 19, Loss: 14.315635
Epoch: 20, Loss: 12.595521


In [190]:
# Prediction
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [194]:
predict(model, "what is the capital city of India?")

delhi


In [195]:
predict(model,"Which element has the atomic number 1?")

hydrogen


In [196]:
predict(model,"What is the hardest natural substance on Earth?")

diamond
